In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Carregar os dados
dataset2_inputs = pd.read_csv('data/dataset2_inputs.csv', delimiter='\t')
test_in = pd.read_csv('data/test_in.csv', delimiter='\t')
test_out = pd.read_csv('data/teste_out.csv', delimiter='\t')

# Pré-processar os dados
X_train = test_in['Text']
y_train = test_out['Label'].apply(lambda x: 1 if x == 'AI' else 0)  # 'AI' = 1, 'Human' = 0

# Tokenização e vetorização dos dados usando CountVectorizer
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X_train_vect = vectorizer.fit_transform(X_train).toarray()

# Funções para regressão logística
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression(X, y, learning_rate=0.01, epochs=100):
    m, n = X.shape
    # Inicializar os pesos
    W = np.zeros(n)
    b = 0

    # Descida do gradiente
    for epoch in range(epochs):
        # Modelo linear
        model = np.dot(X, W) + b
        # Probabilidades previstas
        predictions = sigmoid(model)

        # Cálculo dos gradientes
        dw = (1/m) * np.dot(X.T, (predictions - y))
        db = (1/m) * np.sum(predictions - y)

        # Atualizar os parâmetros
        W -= learning_rate * dw
        b -= learning_rate * db

    return W, b

def predict(X, W, b):
    model = np.dot(X, W) + b
    predictions = sigmoid(model)
    return [1 if p >= 0.5 else 0 for p in predictions]

# Treinar o modelo
W, b = logistic_regression(X_train_vect, y_train, learning_rate=0.01, epochs=100)

# Carregar e pré-processar o dataset2_inputs para previsão
X_input = dataset2_inputs['Text']
X_input_vect = vectorizer.transform(X_input).toarray()

# Prever os resultados usando o modelo treinado
predictions = predict(X_input_vect, W, b)

# Mapear as previsões para os rótulos ('AI' ou 'Human')
predicted_labels = ['AI' if p == 1 else 'Human' for p in predictions]

# Criar o DataFrame de saída
dataset2_inputs['Predicted_Label'] = predicted_labels

# Salvar as previsões num arquivo CSV no mesmo formato de entrada
dataset2_inputs[['ID', 'Predicted_Label']].to_csv('data/dataset2_predictions_logistic-s1.csv', index=False)
